## Academic Paper Translation Pipeline — *o3 Responses API* (v3 🚀)

## 0️⃣ Install dependencies
Correct packages only — `mammoth` is the right name.


In [ ]:
!pip -q install --upgrade mammoth python-docx==1.1.0 tiktoken pypandoc openai tqdm
!apt-get -qq update && apt-get -y -qq install pandoc texlive-xetex


## 1️⃣ Upload your `.docx` and set languages


In [ ]:
from google.colab import files, userdata
uploaded = files.upload()  # choose the Word file
DOCX_PATH = next(iter(uploaded))
print(f"Uploaded: {DOCX_PATH}")

SOURCE_LANG = "Portuguese"  # @param {type:"string"}
TARGET_LANG = "Japanese"     # @param {type:"string"}


## 2️⃣ Utilities


In [18]:
import os, tempfile, textwrap, math
from pathlib import Path
from tqdm.auto import tqdm
import docx, json
import mammoth, tiktoken, pypandoc
from openai import OpenAI

# ---- OpenAI ----
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY') or os.getenv("OPENAI_API_KEY"))
MODEL = "o3"  # o4-mini

# ---- Token helpers ----
try:
    enc = tiktoken.encoding_for_model(MODEL)
except KeyError:
    enc = tiktoken.get_encoding("cl100k_base")

def tokens_of(text: str) -> int:
    return len(enc.encode(text))

# ---- File conversions ----
def docx_to_html(path: str) -> str:
    with open(path, "rb") as f:
        return mammoth.convert_to_html(f).value

def html_to_docx(html: str, out_path: str):
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".html", encoding="utf-8") as tmp:
        tmp.write(html)
        tmp_path = tmp.name
    pypandoc.convert_file(tmp_path, "docx", outputfile=out_path)
    Path(tmp_path).unlink(missing_ok=True)

def docx_to_pdf(docx_path: str, pdf_path: str):
    pypandoc.convert_file(docx_path, "pdf", outputfile=pdf_path)

# ---- Chunking ----
TARGET_CHUNK_TOKENS = 6_000

def chunk_html(html: str, target_tokens: int = TARGET_CHUNK_TOKENS):
    """Yield ~target_tokens blocks, splitting on line boundaries."""
    lines = html.split("\n")
    buf, buf_tokens = [], 0
    for line in lines:
        t = tokens_of(line)
        if buf_tokens + t > target_tokens and buf:
            yield "\n".join(buf)
            buf, buf_tokens = [], 0
        buf.append(line)
        buf_tokens += t
    if buf:
        yield "\n".join(buf)

# ---- Instruction prompt ----
SYSTEM_PROMPT = textwrap.dedent("""
Translate from {source} to {target} using a formal scholarly register.

* Preserve **all** HTML tags, headings, tables, equations, citations and
  reference markers exactly.
* Do **not** add or remove content or change document structure.
* Return **only** valid HTML (no Markdown, no commentary).
""")

# ---- Robust extractor ----
def extract_translated_text(resp) -> str:
    """Return concatenated output_text blocks regardless of resp object style."""
    # Prefer the built‑in Pydantic `model_dump` (OpenAI >= 1.12)
    if hasattr(resp, "model_dump"):
        data = resp.model_dump()
    elif hasattr(resp, "to_dict"):
        data = resp.to_dict()
    else:  # fallback try json
        import json
        data = json.loads(resp.json())
    pieces = []
    for item in data.get("output", []):
        if item.get("type") == "message":
            for part in item.get("content", []):
                if part.get("type") == "output_text":
                    pieces.append(part.get("text", ""))
                    print(part.get("text", ""))
    return "".join(pieces)

# ---- Translator ----
def translate_document(html: str, src: str, tgt: str) -> str:
    translated_chunks = []
    prev_id = None
    chunks = list(chunk_html(html))
    print(f"Total chunks: {len(chunks)} (≈{TARGET_CHUNK_TOKENS} tokens each)")
    for idx, chunk in enumerate(tqdm(chunks, desc="Translating"), start=1):
        resp = client.responses.create(
            model=MODEL,
            instructions=SYSTEM_PROMPT.format(source=src, target=tgt),
            input=chunk,
            previous_response_id=prev_id,
            reasoning={"effort": "high"}
        )
        translated_chunks.append(extract_translated_text(resp))
        prev_id = resp.id
    return "\n".join(translated_chunks)


## 3️⃣ Run the pipeline


In [19]:
html_src = docx_to_html(DOCX_PATH)
print(f"Source tokens: {tokens_of(html_src):,}")

html_trg = translate_document(html_src, SOURCE_LANG, TARGET_LANG)

DOCX_OUT = f"translated_{TARGET_LANG}.docx"
PDF_OUT  = f"translated_{TARGET_LANG}.pdf"

html_to_docx(html_trg, DOCX_OUT)

print("\n🎉  Done!  Download below:")
from google.colab import files as colab_files
colab_files.download(DOCX_OUT)

Source tokens: 156,295
Total chunks: 15 (≈6000 tokens each)


Translating:   0%|          | 0/15 [00:00<?, ?it/s]

<p><strong>ブラジル・日本経済関係の将来展望</strong></p><p>Edson Kenji Kondo</p><p><strong>序論</strong></p><p>本稿はブラジルと日本の外交関係130周年を記念する催しの一環として作成されたものであり、両国間のビジネス関係の将来展望を考察することを目的とする。焦点は、二国間関係の大きな柱の一つを構成するブラジルにおける日本企業の活動に当てられている。</p><p>本稿はエッセイの体裁を取り、インタビュー対象者の経験と知識の恩恵を受けているが、その基盤には著者が日本で企業、NGO、多国間機関および大学において現地勤務した経験、さらに在東京ブラジル大使館勤務の経験がある。</p><p>本稿の作成に際し、日本企業および多国間機関の経営幹部の方々に限られた時間の中で面談の機会を快く提供していただき、大いなるご協力を賜ったことは誠に光栄であった。</p><p>最後に、本稿は日本企業のブラジルにおける事業拡大の展望と、この1世紀以上にわたる実り多いパートナーシップをさらに強化・促進し得る主要要素について考察する。</p><p>多くの人々にとって、日本人移民とブラジル・日本間の130年に及ぶ外交関係は成功の物語である。日系人はブラジルに根を下ろし、日本企業はブラジルに定着した。サンパウロには活発で組織的な日本商工会議所が設置され、ブラジル社会に統合されている。また、JETRO、JICA、JBIC など多数の日本機関がブラジルで活動しているほか、多様な形態の日系コミュニティ団体が存在する。</p><p>20世紀初頭の黎明期に日本当局の奨励により最初の移民が渡航した際、ブラジルはほとんど楽園のように描写されていた。そこでは散策中に地面に落ちている鶏卵を拾え、森には多種多様な果樹が自生しているという説明であった。厳密にいえば今日でもブラジル内陸部の小規模農地で類似の状況が見られるが、到着後に多くの家族が半奴隷的な境遇に置かれた現実はその描写とかけ離れていた。</p><p>異なる慣習と言語を持つ新天地に適応する自然な困難に加え、移民とブラジルの住民および当局との関係は必ずしも平穏ではなかった。</p><p>1908年の日本船笠戸丸到着から最初の数十年で、ブラジル国内ではナショナリズムと外国人排斥が強まり、国内メディア

BadRequestError: Error code: 400 - {'error': {'message': 'One of "input" or "previous_response_id" must be provided.', 'type': 'invalid_request_error', 'param': None, 'code': 'missing_required_parameter'}}